# 아마존 세일즈 데이터 EDA - 보고서 작성용

## Introduction

> 앞선 3주 동안 약 **1400개** 행의 소규모 데이터를 분석하며, 기본적인 통계기법과 클러스터링에 대해 파악함.   
> 이번 EDA의 목적은 2023년의 인도, 영국, 미국, 캐나다의 **대규모** 마켓 세일즈 데이터(~150만개)를 분석함에 있음.

|#| Table of Contents | Finished |
|:--|:--:|:--:|
|1| Install Necessary Packages & Load Data | &check; |
|2| Acknowledge Characteristics of Data | &cross;|


&copy; 2024 Yoori Choi <it.glasschoi@gmail.com>
* * *

## 1. Install Necessary Packages & Load Data

In [1]:
# Import Necessary Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import warnings

# from tqdm import tqdm_notebook
warnings.simplefilter(action='ignore')
%matplotlib inline

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")

print("Path to dataset files:", path)

In [4]:
# Load Data
origin_india_df = pd.read_csv('../../data/amazon.csv')
india_df = pd.read_csv('../../data/amazon_india.csv')
canada_df = pd.read_csv('../../data/amazon_canada.csv')
uk_df = pd.read_csv('../../data/amazon_uk.csv')
usa_df = pd.read_csv('../../data/amazon_usa.csv')

In [5]:
india_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   version https://git-lfs.github.com/spec/v1  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes
